# 케라스로 AlexNet 만들기

이 노트북에서 [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks)과 비슷한 심층 합성곱 신경망으로 [Oxford Flowers](http://www.robots.ox.ac.uk/~vgg/data/flowers/17/) 데이터셋의 꽃을 17개의 카테고리로 분류하겠습니다.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/dl-illustrated/blob/master/notebooks/10-2.alexnet_in_keras.ipynb)

#### 라이브러리를 적재합니다.

In [1]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

#### 데이터를 적재하고 전처리합니다.

원서 노트북은 tflearn을 사용해 oxflower17 데이터셋을 다운로드합니다. 이 라이브러리는 텐서플로 2와 호환되지 않습니다. 여기에서는 사전에 tflearn으로 다운받은 데이터를 다운로드하여 사용합니다.

이 데이터셋에 대한 자세한 내용은 http://www.robots.ox.ac.uk/~vgg/data/flowers/17/ 을 참고하세요.

In [2]:
!rm oxflower17*
!wget https://bit.ly/36QytdH -O oxflower17.npz

rm: cannot remove 'oxflower17*': No such file or directory
--2021-08-21 13:17:17--  https://bit.ly/36QytdH
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://onedrive.live.com/download?cid=822579D69D2DC3B5&resid=822579D69D2DC3B5!597859&authkey=AGd0CpvKFkK8GtE [following]
--2021-08-21 13:17:17--  https://onedrive.live.com/download?cid=822579D69D2DC3B5&resid=822579D69D2DC3B5!597859&authkey=AGd0CpvKFkK8GtE
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://57ucia.bl.files.1drv.com/y4mUhPdcQlUGI4Asclf-kTSiJd9xqhVdD2k52tazj5OESl0aN1ZuABqmd3K42eOR31yTxCFHzqq9beBzlEU8Sxk1aaerRsHblaMR1ufzAQC7nCx5-d4ejgiQ216SbnXEzim16gkpNMTcYFjTIAbsu9lshhvf7Um6GwRE0M3CfmppZcHGTLhMOzTehxK8egZJ7hXn3qDJkt

In [3]:
import numpy as np

data = np.load('oxflower17.npz')
X = data['X']
Y = data['Y']

In [4]:
X.shape, Y.shape

((1360, 224, 224, 3), (1360, 17))

#### 신경망 모델을 만듭니다.

In [5]:
model = Sequential()

model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))

model.add(Dense(17, activation='softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 10, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 256)         5

#### 모델을 설정합니다.

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 훈련!

In [8]:
model.fit(X, Y, batch_size=64, epochs=100, verbose=1, validation_split=0.1, shuffle=True)

Epoch 1/100
20/20 [==============================] - 34s 86ms/step - loss: 4.5239 - accuracy: 0.1871 - val_loss: 6.7543 - val_accuracy: 0.1324
Epoch 2/100
20/20 [==============================] - 1s 53ms/step - loss: 3.1264 - accuracy: 0.2876 - val_loss: 3.6883 - val_accuracy: 0.0956
Epoch 3/100
20/20 [==============================] - 1s 54ms/step - loss: 2.6828 - accuracy: 0.3129 - val_loss: 4.4686 - val_accuracy: 0.2059
Epoch 4/100
20/20 [==============================] - 1s 53ms/step - loss: 2.5200 - accuracy: 0.3456 - val_loss: 4.0328 - val_accuracy: 0.2132
Epoch 5/100
20/20 [==============================] - 1s 55ms/step - loss: 2.4953 - accuracy: 0.3358 - val_loss: 3.4560 - val_accuracy: 0.1985
Epoch 6/100
20/20 [==============================] - 1s 54ms/step - loss: 2.1424 - accuracy: 0.3979 - val_loss: 3.6366 - val_accuracy: 0.2059
Epoch 7/100
20/20 [==============================] - 1s 54ms/step - loss: 2.0597 - accuracy: 0.4199 - val_loss: 3.3827 - val_accuracy: 0.2426
Epoch